In [1]:
import torch
import numpy as np
import sys
sys.path.append('./fcn_opt')
sys.path.append('../')

from ttgo import TTGO
from test_fcns import sine_nD 
from fcn_plotting_utils import plot_surf, plot_contour

%load_ext autoreload
np.set_printoptions(precision=2)
torch.set_printoptions(precision=2)

%autoreload 2

#### Define the cost and the correpsonding pdf function

In [6]:
n=3
pdf, cost =  sine_nD(n=n,alpha=1) # n>=4

#### Define the domain and the discretization

In [7]:
# Define the domain of the function

L = 2 # [-L,L]^n is the domain of the function

# domain of task params: domain of coefficients a and b in Rosenbrock_4D
# Note: a should in (-sqrt(L), sqrt(L)) 
domain_task = [torch.linspace(-np.sqrt(L),np.sqrt(L),1000)]+[torch.linspace(50,150,1000)] 
# domain of decison varibales
domain_decision = [torch.linspace(-L,L,1000)]*(n)
domain = domain_task+domain_decision

### Fit the TT-Model

In [8]:
ttgo = TTGO(domain=domain,pdf=pdf,cost=cost)

In [9]:
ttgo.cross_approximate(rmax=50, nswp=5, kickrank=3, eps=1e-6)

cross device is cpu
Cross-approximation over a 5D domain containing 1e+15 grid points:
Note: The algorithm converges as the ratio tt-new-norm/tt-old-norm settles to 1. For TTGO, the convergence is not important, just keep iterating as long as the ratio > 1
iter: 0 | tt-new-norm/tt-old-norm: 5.474e-01 | time:   0.0154 | largest rank:   1
iter: 1 | tt-new-norm/tt-old-norm: 1.005e+00 | time:   0.0536 | largest rank:   4
iter: 2 | tt-new-norm/tt-old-norm: 1.000e+00 | time:   0.1157 | largest rank:   7
iter: 3 | tt-new-norm/tt-old-norm: 1.000e+00 | time:   0.2205 | largest rank:  10
iter: 4 | tt-new-norm/tt-old-norm: 1.000e+00 | time:   0.3952 | largest rank:  13 <- max_iter was reached: 5
Did 2115000 function evaluations, which took 0.2502s (8.453e+06 evals/s)



In [10]:
sites_task = torch.tensor([0,1,2]) # sites to be conditioned upon (task-params)
ttgo.set_sites(sites_task)
ttgo.round(eps=1e-9)
print("TT-Rank: ", ttgo.tt_model.ranks_tt)

TT-Rank:  tensor([1, 5, 7, 6, 5, 1])


#### Specify task parameters

In [11]:
x_task = torch.tensor([1]*3).view(1,-1) #given task-parameters

### Sample from TT-Model

In [12]:
n_samples_tt = 50

samples, samples_idx = ttgo.sample(n_samples=n_samples_tt, x_task=x_task, alpha=0.9, norm=1) 


### Choose the best sample as an estimate for optima (initial guess)

In [13]:
best_estimate = ttgo.choose_best_sample(samples)
top_k_estimate = ttgo.choose_top_k_sample(samples,k=10) # for multiple solutions

##### Fine-tune the estimate using gradient-based optimization

In [14]:
ttgo_optimized,_ = ttgo.optimize(best_estimate)

ttgo_optimized_k = 1*top_k_estimate
for i, x in enumerate(ttgo_optimized_k):
    ttgo_optimized_k[i],_ = ttgo.optimize(x)


In [15]:
print("PDF at the estimated point(initial guess): ", pdf(best_estimate))
print("PDF at the TTGO Optima: ", pdf(ttgo_optimized))

PDF at the estimated point(initial guess):  tensor([0.09])
PDF at the TTGO Optima:  tensor([0.09])


In [16]:
print("Estimated Optima: ", best_estimate)
print("Optima from TTGO: ", ttgo_optimized)

Estimated Optima:  tensor([[ 1.00, 50.00,  1.00, -2.00, -2.00]])
Optima from TTGO:  tensor([[ 1.00, 50.00,  1.00, -2.00, -2.00]])


### Compare with samples from uniform distribution (initialization)

In [17]:
n_samples_rand = 10*n_samples_tt 
samples_rand,_ = ttgo.sample_random(n_samples=n_samples_rand,x_task=x_task)

rand_best_estimate = ttgo.choose_best_sample(samples_rand)
rand_optimized,_ = ttgo.optimize(rand_best_estimate)

print("PDF at the estimated point(initial guess): ", pdf(rand_best_estimate))
print("PDF at the Optima: ", pdf(rand_optimized))

PDF at the estimated point(initial guess):  tensor([0.09])
PDF at the Optima:  tensor([0.09])
